# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [2]:
from datasets import load_dataset

# Using dair-ai/emotion because I want to try to make a text classifier with a classification head with more than 2 outputs.
# This dataset has 6 classes
ds = load_dataset("dair-ai/emotion", "split")

In [3]:
from transformers import AutoTokenizer

# Using distilbert/distilbert-base-uncased because it is good at sequence classification
foundation_model_name = "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(foundation_model_name)

def preprocess(data):
    return tokenizer(data['text'], truncation=True)

tokenized_ds = {}
for k, v in ds.items():
    tokenized_ds[k] = v.map(preprocess, batched=True)

In [4]:
from transformers import AutoModelForSequenceClassification

id2label = {0: "joy", 1: "sadness", 2: "anger", 3: "fear",  4: "love", 5: "suprise"}
label2id = {"joy": 0, "sadness": 1, "anger": 2, "fear": 3, "love": 4, "suprise": 5}

base_model = AutoModelForSequenceClassification.from_pretrained(
    foundation_model_name,
    num_labels=6,
    id2label=id2label,
    label2id=label2id
)
base_model.to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [5]:
import os
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

base_trainer = Trainer(
    model=base_model,
    args=TrainingArguments(
        output_dir=os.path.join(os.getcwd(), "data", "before_finetune"),
        per_device_eval_batch_size=16,
    ),
    eval_dataset=tokenized_ds['validation'],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

base_eval = base_trainer.evaluate()

/var/folders/1w/pwvs5lks2fjg2q9rq83vw1r00000gn/T/ipykernel_63776/2983880928.py:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  base_trainer = Trainer(


In [6]:
import pandas as pd


base_results = base_trainer.predict(tokenized_ds["validation"])
df_base = pd.DataFrame(
    {
        "text": [item["text"] for item in tokenized_ds["validation"]],
        "predictions": base_results.predictions.argmax(axis=1),
        "labels": base_results.label_ids,
    }
)
df_base["predictions"] = df_base["predictions"].map(lambda x: id2label[x])
df_base["labels"] = df_base["labels"].map(lambda x: id2label[x])

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [7]:
from peft import get_peft_model, LoraConfig, AutoPeftModelForSequenceClassification

# Using LoRA peft technique because it works well for classification tasks and efficient to train
config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_lin", "k_lin"],
    bias="none",
    task_type="SEQ_CLS"
)

lora_model = get_peft_model(base_model, config)
lora_model.to(device)

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): DistilBertSdpaAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=76

In [8]:
lora_trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir=os.path.join(os.getcwd(), "data", "during_finetune"),
        learning_rate=1e-3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        eval_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=5,
        weight_decay=0.01,
        load_best_model_at_end=True
    ),
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['validation'],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

lora_trainer.train()

/var/folders/1w/pwvs5lks2fjg2q9rq83vw1r00000gn/T/ipykernel_63776/3237740506.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  lora_trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.383800,0.258494,0.907500
2,0.277800,0.220851,0.921000
3,0.205400,0.210242,0.930500
4,0.163600,0.186323,0.934000
5,0.130200,0.181397,0.932500


TrainOutput(global_step=5000, training_loss=0.2555417922973633, metrics={'train_runtime': 199.394, 'train_samples_per_second': 401.216, 'train_steps_per_second': 25.076, 'total_flos': 989542873021824.0, 'train_loss': 0.2555417922973633, 'epoch': 5.0})

In [9]:
lora_model.save_pretrained(os.path.join(os.getcwd(), "models", "distilbert-emotion"))

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [10]:
lora_model = AutoPeftModelForSequenceClassification.from_pretrained(os.path.join(os.getcwd(), "models", "distilbert-emotion"), num_labels=6)
lora_model.to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): DistilBertSdpaAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=76

In [11]:
inference_trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir=os.path.join(os.getcwd(), "data", "after_finetune"),
        per_device_eval_batch_size=16,
        report_to="none"
    ),
    eval_dataset=tokenized_ds['validation'],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

lora_eval = inference_trainer.evaluate()

/var/folders/1w/pwvs5lks2fjg2q9rq83vw1r00000gn/T/ipykernel_63776/4044527155.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  inference_trainer = Trainer(


In [12]:
lora_results = inference_trainer.predict(tokenized_ds["validation"])
df_lora = pd.DataFrame(
    {
        "text": [item["text"] for item in tokenized_ds["validation"]],
        "predictions": lora_results.predictions.argmax(axis=1),
        "labels": lora_results.label_ids,
    }
)
df_lora["predictions"] = df_lora["predictions"].map(lambda x: id2label[x])
df_lora["labels"] = df_lora["labels"].map(lambda x: id2label[x])

In [13]:
pd.set_option("display.max_colwidth", None)

print(f"The base model had an accuracy of {(base_eval['eval_accuracy'] * 100):.2f}%")
df_base # don't even need to filter for failed predictions on the base model

The base model had an accuracy of 8.90%


,text,predictions,labels
0,im feeling quite sad and sorry for myself but ill snap out of it soon,anger,joy
1,i feel like i am still looking at a blank canvas blank pieces of paper,anger,joy
2,i feel like a faithful servant,anger,anger
3,i am just feeling cranky and blue,anger,fear
4,i can have for a treat or if i am feeling festive,anger,sadness
...,...,...,...
1995,im having ssa examination tomorrow in the morning im quite well prepared for the coming exam and somehow i feel numb towards exam because in life there is much more important things than exam,anger,joy
1996,i constantly worry about their fight against nature as they push the limits of their inner bodies for the determination of their outer existence but i somehow feel reassured,anger,sadness
1997,i feel its important to share this info for those that experience the same thing,anger,sadness
1998,i truly feel that if you are passionate enough about something and stay true to yourself you will succeed,anger,sadness


In [14]:
print(f"The lora model had an accuracy of {(lora_eval['eval_accuracy'] * 100):.2f}%")
df_lora[df_lora["predictions"] != df_lora["labels"]].head(10)

The lora model had an accuracy of 93.25%


,text,predictions,labels
54,i feel a special draw toward and awed admiration for the firefighters who led the charge into the towers when everyone else was rushing out,suprise,sadness
60,i miss our talks our cuddling our kissing and the feelings that you can only share with your beloved,sadness,anger
91,i feel like the people i know are really generous and i have my needs met,anger,sadness
115,i went from feeling helpless to powerful,love,joy
118,when i was doing research a few months ago,love,fear
136,i feel such gratitude for the generous gifts we received on our wedding day over years ago,sadness,anger
164,i have stayed at heritage christian because of the fulfillment that i feel in doing christ s work in action by being the hands the eyes the legs and the voice of supporting the individuals that i have been blessed to know and support,anger,sadness
177,im sure much of the advantage is psychological the feeling ive out clevered the competition who are now hopelessly burdened with their big chainring jump,sadness,joy
195,i also remember feeling like all eyes were on me all the time and not in a glamorous way and i hated it,joy,sadness
200,i really have much of a clue how my ex actually feels or felt about anything really except that he hated it when i didnt screw the lids back on jars in the kitchen,joy,fear
